In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nest-competition/eligibilities.txt
/kaggle/input/nest-competition/facilities_cleaned.csv
/kaggle/input/nest-competition/eligibilities_cleaned.csv
/kaggle/input/nest-competition/drop_withdrawals_cleaned.csv
/kaggle/input/nest-competition/reported_events.txt
/kaggle/input/nest-competition/facilities.txt
/kaggle/input/nest-competition/usecase_2_.csv
/kaggle/input/nest-competition/drop_withdrawals.txt


In [2]:
import seaborn as sb
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/kaggle/input/nest-competition/usecase_2_.csv')
df = df.rename(columns={"NCT Number": "nct_id"})
facilities = pd.read_csv('/kaggle/input/nest-competition/facilities.txt', sep='|')
facilities.head()

,id,nct_id,status,name,city,state,zip,country
0,39182239,NCT02696421,RECRUITING,Imperial College London Diabetes Centre,Abu Dhabi,NaN,48338,United Arab Emirates
1,39182240,NCT01324414,NaN,Mayo Clinic,Phoenix,Arizona,85054,United States
2,39182241,NCT02595814,NaN,Novartis Investigative Site,San Diego,California,92103,United States
3,39182242,NCT02595814,NaN,Novartis Investigative Site,San Francisco,California,94143-0780,United States
4,39182243,NCT02595814,NaN,Novartis Investigative Site,Chicago,Illinois,60611,United States


In [4]:
df.shape

(68960, 26)

In [5]:
facilities.shape

(3085464, 8)

In [6]:
# Perform an inner merge to keep only the rows in drop_withdrawals where nct_id is in df
facilities = facilities.merge(df[['nct_id']], on='nct_id', how='inner')
facilities.shape

(398949, 8)

In [7]:
facilities.drop_duplicates(keep="first", inplace=True)
facilities.shape

(398949, 8)

In [8]:
facilities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398949 entries, 0 to 398948
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       398949 non-null  int64 
 1   nct_id   398949 non-null  object
 2   status   755 non-null     object
 3   name     380534 non-null  object
 4   city     398949 non-null  object
 5   state    227890 non-null  object
 6   zip      343198 non-null  object
 7   country  398949 non-null  object
dtypes: int64(1), object(7)
memory usage: 24.4+ MB


In [9]:
df['Study Status'].unique()

array(['COMPLETED'], dtype=object)

In [10]:
facilities['status'].unique()

array([nan, 'NOT_YET_RECRUITING', 'RECRUITING', 'ENROLLING_BY_INVITATION',
       'COMPLETED', 'ACTIVE_NOT_RECRUITING', 'WITHDRAWN', 'TERMINATED'],
      dtype=object)

In [11]:
facilities = facilities.drop(['id', 'status'], axis='columns')

In [12]:
facilities.head()

,nct_id,name,city,state,zip,country
0,NCT02889796,NaN,Nowa Sól,NaN,NaN,Poland
1,NCT02889796,NaN,Poznan,NaN,NaN,Poland
2,NCT02889796,NaN,Poznań,NaN,NaN,Poland
3,NCT02889796,NaN,Tomaszów Lubelski,NaN,NaN,Poland
4,NCT02889796,NaN,Toruń,NaN,NaN,Poland


In [13]:
facilities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398949 entries, 0 to 398948
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   nct_id   398949 non-null  object
 1   name     380534 non-null  object
 2   city     398949 non-null  object
 3   state    227890 non-null  object
 4   zip      343198 non-null  object
 5   country  398949 non-null  object
dtypes: object(6)
memory usage: 18.3+ MB


In [14]:
# facilities.iloc[50:100]

In [16]:
# Step 1: Compute modes
zip_mode_per_city = facilities.dropna(subset=['zip']).groupby('city')['zip'].agg(lambda x: x.mode()[0])
state_mode_per_city = facilities.dropna(subset=['state']).groupby('city')['state'].agg(lambda x: x.mode()[0])
country_mode_per_state = facilities.dropna(subset=['country']).groupby('state')['country'].agg(lambda x: x.mode()[0])
name_mode_per_nct_id = facilities.dropna(subset=['name']).groupby('nct_id')['name'].agg(lambda x: x.mode()[0])

# Step 2: Replace mismatched or NaN values
# Replace all mismatched or NaN state values for a given city with the mode state
facilities['zip'] = facilities.apply(lambda row: zip_mode_per_city[row['city']] if row['city'] in zip_mode_per_city else row['zip'], axis=1)

# Replace all mismatched or NaN state values for a given city with the mode state
facilities['state'] = facilities.apply(lambda row: state_mode_per_city[row['city']] if row['city'] in state_mode_per_city else row['state'], axis=1)

# Replace all mismatched or NaN country values for a given state with the mode country
facilities['country'] = facilities.apply(lambda row: country_mode_per_state[row['state']] if row['state'] in country_mode_per_state else row['country'], axis=1)

# Replace only NaN name values for a given nct_id with the mode name
facilities['name'] = facilities['name'].fillna(facilities['nct_id'].map(name_mode_per_nct_id))

In [17]:
facilities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398949 entries, 0 to 398948
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   nct_id   398949 non-null  object
 1   name     382663 non-null  object
 2   city     398949 non-null  object
 3   state    380252 non-null  object
 4   zip      395949 non-null  object
 5   country  398949 non-null  object
dtypes: object(6)
memory usage: 18.3+ MB


In [20]:
# Step 1: Pre-calculate modes for all relevant columns
mode_df = facilities.dropna(subset=['zip', 'city', 'name', 'state', 'country']).groupby('nct_id').agg({
    'zip': lambda x: x.mode()[0] if not x.mode().empty else None,
    'city': lambda x: x.mode()[0] if not x.mode().empty else None,
    'name': lambda x: x.mode()[0] if not x.mode().empty else None,
    'state': lambda x: x.mode()[0] if not x.mode().empty else None,
    'country': lambda x: x.mode()[0] if not x.mode().empty else None
}).rename(columns={
    'zip': 'mode_zip',
    'city': 'mode_city',
    'name': 'mode_name',
    'state': 'mode_state',
    'country': 'mode_country'
})

# Step 2: Count unique values efficiently
unique_counts = facilities.groupby('nct_id').agg({
    'name': lambda x: x.nunique(dropna=False),
    'state': lambda x: x.nunique(dropna=False),
    'zip': lambda x: x.nunique(dropna=False),
    'city': 'nunique',
    'country': 'nunique'
}).rename(columns={
    'name': 'unique_name_count',
    'state': 'unique_state_count',
    'zip': 'unique_zip_count',
    'city': 'unique_city_count',
    'country': 'unique_country_count'
})

# Step 3: Merge modes and counts into a single DataFrame
final_counts = mode_df.join(unique_counts, on='nct_id').reset_index()

# Output: Check the final DataFrame
print(final_counts.head())

# Optional: Save the result to a file
# final_counts.to_csv('nct_id_counts_modes.csv', index=False)


        nct_id mode_zip   mode_city  \
0  NCT00323141     9000       Ghent   
1  NCT00737893    21287   Baltimore   
2  NCT00862433    20892    Bethesda   
3  NCT00900809    15213  Pittsburgh   
4  NCT01066234    03080       Seoul   

                                       mode_name         mode_state  \
0                      University Hospital Ghent    Oost-Vlaanderen   
1                         Johns Hopkins Hospital           Maryland   
2  National Institutes of Health Clinical Center           Maryland   
3     UPMC Cancer Center - Hillman Cancer Center       Pennsylvania   
4                         Samsung Medical Center  Seoul Teugbyeolsi   

         mode_country  unique_name_count  unique_state_count  \
0             Belgium                  1                   1   
1       United States                  1                   1   
2       United States                  1                   1   
3       United States                  1                   1   
4  Korea, Republic

In [22]:
final_counts.head(50)

,nct_id,mode_zip,mode_city,mode_name,mode_state,mode_country,unique_name_count,unique_state_count,unique_zip_count,unique_city_count,unique_country_count
0,NCT00323141,9000,Ghent,University Hospital Ghent,Oost-Vlaanderen,Belgium,1,1,1,1,1
1,NCT00737893,21287,Baltimore,Johns Hopkins Hospital,Maryland,United States,1,1,1,1,1
2,NCT00862433,20892,Bethesda,National Institutes of Health Clinical Center,Maryland,United States,1,1,1,1,1
3,NCT00900809,15213,Pittsburgh,UPMC Cancer Center - Hillman Cancer Center,Pennsylvania,United States,1,1,1,1,1
4,NCT01066234,03080,Seoul,Samsung Medical Center,Seoul Teugbyeolsi,"Korea, Republic of",1,1,1,1,1
5,NCT01113515,02903,Chennai,Deenanath Mangeshkar Hospital & Research Center,Maharashtra,India,5,4,5,5,3
6,NCT01229631,1105 AZ,Amsterdam,Academic Centre for Dentistry,Alabama,Germany,3,3,3,3,3
7,NCT01267448,60611,Chicago,John Stroger Hospital of Cook County,Illinois,United States,1,1,1,1,1
8,NCT01288573,00168,Amsterdam,Investigational Site Number 11,Alabama,Italy,27,23,27,27,13
9,NCT01290055,30030,Decatur,The Hope Clinic of Emory Vaccine Center,Georgia,United States,1,1,1,1,1


In [23]:
facilities[facilities['nct_id'] == 'NCT01113515']

,nct_id,name,city,state,zip,country
210867,NCT01113515,VA New England Health Care Division,Providence,Rhode Island,02903,United States
210868,NCT01113515,S.L. Raheja Hospital,Mumbai,Maharashtra,400012,India
210869,NCT01113515,Deenanath Mangeshkar Hospital & Research Center,Pune,Maharashtra,411001,India
210870,NCT01113515,M V Hospital for Diabetes Pvt. Ltd.,Chennai,Tamil Nadu,600006,India
210871,NCT01113515,Hospital Kuala Lumpur,Kuala Lumpur,Wilayah Persekutuan,59100,Malaysia


In [24]:
final_counts.to_csv('facilities_cleaned.csv', index=False)